In [16]:
# exports from an SQL query to an excel sheet suitable for iServer

import json
import os
import sqlite3
import pandas as pd

from pathlib import Path
from datetime import datetime

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "export_output_files/iServer/"
output_filename = "conceptual_relations_" + time_string + ".xlsx"


In [17]:
sql_query = """
SELECT
    l_cecer.ConceptualEntityConceptualEntity_RelatedHashKey
  , l_cecer.ConceptualEntityHashKey
  , bv_ce.ConceptualEntityName AS Source_Entity_Name
  , l_cecer.ConceptualEntity_RelatedHashKey
  , bv_cer.ConceptualEntityName AS Related_Entity_Name
  , s_cecer.label_forward
  , s_cecer.label_reverse
FROM
  rv_l_ConceptualEntityConceptualEntity_Related l_cecer
  INNER JOIN rv_s_ConceptualEntityConceptualEntity_Related s_cecer ON (
    l_cecer.ConceptualEntityConceptualEntity_RelatedHashKey = s_cecer.ConceptualEntityConceptualEntity_RelatedHashKey
    AND s_cecer.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_ConceptualEntityConceptualEntity_Related AS z
      WHERE z.ConceptualEntityConceptualEntity_RelatedHashKey = s_cecer.ConceptualEntityConceptualEntity_RelatedHashKey
    )
  )
  LEFT JOIN bv_ConceptualEntity bv_ce ON (
    bv_ce.ConceptualEntityHashKey = l_cecer.ConceptualEntityHashKey
  )
  LEFT JOIN bv_ConceptualEntity bv_cer ON (
    bv_cer.ConceptualEntityHashKey = l_cecer.ConceptualEntity_RelatedHashKey
  )
WHERE
  s_cecer.isDeleted = 0
ORDER BY
  bv_ce.ConceptualEntityName
;
"""

df = pd.read_sql_query(sql_query, conn)

#df

In [18]:
writer = pd.ExcelWriter(os.path.join(output_path, output_filename))
df.to_excel(writer)#, sheet_name='bar'
writer.close()